## Installation

In [ ]:
# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 21.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.28-cp310-cp310-manylinux_2_35_x86_64.whl size=8739347 sha256=bda5d3294f4372501131068e193215587f8b1591bce4863c0aa0798f95bf4833
  Stored in directory: /root/.cache/pip/wheels/93/6e/a9/478cce089dc2a082bdcffe468a1c65465c91b25d911b30da82
Successfully built llama-cpp-python


In [ ]:
!pip install -q huggingface_hub

## Imports

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

# Llama2

### Model

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf" # the model is in gguf format

In [ ]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [ ]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096 # Context window
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
llama2_prompt_template = """<s>[INST]<<SYS>>
{system_message}
<</SYS>>

{user_message}[/INST]"""

## Self-consistency

In self-consistency, we generate multiple answers to the same question and pick the answer that is repeated the most across these occurrences. This is particularly valuable for factual questions.

In [ ]:
system_message = """
You are an assistant tasked to answer queries on financial information.
Do not repeat the question. Only answer the question presented by the user.
"""

In [ ]:
answers_template = """
Context:
{context}
===
Using the context above generate {num_answers} distinct answers to the following question:
Question:
{question}.

Arrange your answers in numbered bullet points.
Present only the answers in bullet points.
"""

Here is an extract from the [Tesla 2022 10-K](https://www.sec.gov/Archives/edgar/data/1318605/000095017023001409/tsla-20221231.htm) statement that will be used as context for this demonstration.

In [ ]:
tesla_annual_report_context ="""
In 2022, we recognized total revenues of $81.46 billion, respectively, representing an increase of $27.64 billion, compared to the prior year.
We continue to ramp production, build new manufacturing capacity and expand our operations to enable increased deliveries and deployments of our products and further revenue growth.
"""

In [ ]:
factual_question = "What was the increase in annual revenue in 2022 compared to 2021?"

In [ ]:
answers_prompt = llama2_prompt_template.format(
    system_message=system_message,
    user_message=answers_template.format(
        context=tesla_annual_report_context,
        question=factual_question,
        num_answers=3
    )
)

In [ ]:
response = lcpp_llm(
    prompt=answers_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False # do not return the prompt
)

Llama.generate: prefix-match hit


In [ ]:
print(response["choices"][0]["text"])

  Sure, here are three distinct answers to the question "What was the increase in annual revenue in 2022 compared to 2021?" based on the provided context:

1. The increase in annual revenue in 2022 compared to 2021 was $27.64 billion.
2. Our annual revenues grew by 38% from 2021 to 2022, totaling $81.46 billion.
3. We experienced a significant increase in revenue of $27.64 billion between 2021 and 2022, bringing our total annual revenues to $81.46 billion.


In [ ]:
factual_answers = response["choices"][0]["text"]

In [ ]:
consistency_template = """
Here are {num_answers} answers to the question mentioned below:
Question:
{question}
Answers:
{answers}

Observe the answers mentioned above and choose the answer that occurs most.
Present only the most frequent solution in the following format.
Final Answer:
"""

In [ ]:
consistency_prompt = llama2_prompt_template.format(
    system_message=system_message,
    user_message=consistency_template.format(
        num_answers=3,
        question=factual_question,
        answers=factual_answers
    )
)

In [ ]:
response = lcpp_llm(
    prompt=consistency_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False # do not return the prompt
)

Llama.generate: prefix-match hit


In [ ]:
print(response["choices"][0]["text"])

  Sure! Based on the provided context, the most frequent answer is:

Final Answer: The increase in annual revenue in 2022 compared to 2021 was $27.64 billion.


## Tree-of-Thought

Tree-of-thought prompting is a generalization of chain-of-thought prompting where the model is prompted to take multiple reasoning paths. This forces the LLM into a deliberate reasoning mode.

In [ ]:
solutions_template = """
Generate {num_solutions} distinct solutions for the following problem:
Problem:
{problem}.
--

Consider the following factors in coming up with your solutions.
Factors:
{factors}

Present the solutions in numbered bullet points. Present only the solutions.
"""

In [ ]:
climate_problem = "Reduce the impact of climate change on the occurrence of extreme events in the Earth's atmosphere."

climate_factors = """
1. Renewable Energy Transition
2. Reforestation
3. Sustainable Agricultural Practises
4. Carbon capture and storage
5. Climate-resilient infrastructure
6. Circular economy practises
"""

In [ ]:
solutions_prompt = llama2_prompt_template.format(
    system_message='',
    user_message=solutions_template.format(
        num_solutions=3,
        problem=climate_problem,
        factors=climate_factors
    )
)

In [ ]:
response = lcpp_llm(
    prompt=solutions_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False # do not return the prompt
)

Llama.generate: prefix-match hit


In [ ]:
print(response["choices"][0]["text"])

  Sure, here are three distinct solutions for reducing the impact of climate change on extreme events:

Solution 1: Renewable Energy Transition

• Accelerate the transition to renewable energy sources such as wind and solar power to reduce dependence on fossil fuels and lower carbon emissions.
• Invest in research and development of new technologies to improve efficiency and affordability of renewables.
• Implement policies to promote the adoption of renewables, such as tax credits or feed-in tariffs.

Solution 2: Reforestation and Sustainable Land Use

• Restore degraded lands and forests to their natural state through reforestation efforts.
• Implement sustainable agricultural practices that promote soil health, biodiversity, and carbon sequestration.
• Promote agroforestry techniques that integrate trees into farming systems.

Solution 3: Climate-Resilient Infrastructure and Circular Economy Practices

• Build climate-resilient infrastructure such as sea walls, levees, and green roo

In [ ]:
climate_solutions = response["choices"][0]["text"]

In [ ]:
evaluation_template = """
For the following problem: {problem}, evaluate each solution in the following proposed solutions: \n{solutions}\n.

Analyze pros, cons, feasibility, and probability of success for each solution.
Present your evaluations of each solution.
"""

In [ ]:
evaluations_prompt = llama2_prompt_template.format(
    system_message='',
    user_message=evaluation_template.format(
        problem=climate_problem,
        solutions=climate_solutions
    )
)

In [ ]:
response = lcpp_llm(
    prompt=evaluations_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False # do not return the prompt
)

Llama.generate: prefix-match hit


In [ ]:
climate_proposal_evaluations = response["choices"][0]["text"]

In [ ]:
print(climate_proposal_evaluations)

  Sure! Here are my evaluations of the three proposed solutions to reduce the impact of climate change on extreme events:

Solution 1: Renewable Energy Transition

Pros:

* Reduces dependence on fossil fuels and lowers carbon emissions, which contribute to climate change.
* Can create jobs and stimulate economic growth in the renewable energy sector.
* Improves air quality and public health by reducing pollution from fossil fuel combustion.

Cons:

* Requires significant investment in infrastructure and technology development.
* May face resistance from fossil fuel industries and communities reliant on them.
* Intermittent nature of renewable energy sources can create power grid stability challenges.

Feasibility: High probability of success, as many countries have already made significant progress in transitioning to renewable energy sources. However, the pace of transition needs to be accelerated to meet climate change mitigation goals.

Probability of Success: 8/10

Solution 2: Refo

In [ ]:
ranking_template = """
For the following problem: {problem}, rank the solutions presented in the following evaluations: \n{evaluations}\n.
Pick most promising solution and present implementation strategies and methods to handle potential obstacles for this solution.
"""

In [ ]:
ranking_prompt = llama2_prompt_template.format(
    system_message='',
    user_message=ranking_template.format(
        problem=climate_problem,
        evaluations=climate_proposal_evaluations
    )
)

In [ ]:
response = lcpp_llm(
    prompt=ranking_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False # do not return the prompt
)

Llama.generate: prefix-match hit


In [ ]:
climate_proposal_rankings = response["choices"][0]["text"]

In [ ]:
print(climate_proposal_rankings)

  Based on the evaluations provided, the most promising solution for reducing the impact of climate change on extreme events is the transition to renewable energy sources. This solution has a high probability of success, as many countries have already made significant progress in transitioning to renewable energy sources, and it addresses multiple aspects of the climate crisis, including greenhouse gas emissions and air pollution.

To implement this solution effectively, the following implementation strategies can be considered:

1. Increase investment in renewable energy infrastructure: Governments and private investors should increase their investment in renewable energy technologies such as wind, solar, hydroelectric, and geothermal power. This will help to reduce our reliance on fossil fuels and lower carbon emissions.
2. Implement policies to support the transition: Governments can implement policies such as tax credits, feed-in tariffs, and renewable portfolio standards to incent

# Mistral 7b

### Model

In [ ]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q5_K_M.gguf" # the model is in gguf format

In [ ]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q5_K_M.gguf:   0%|          | 0.00/5.13G [00:00<?, ?B/s]

In [ ]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096 # Context window
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
mistral_prompt_template = """<s>[INST]{prompt}[/INST]"""

## Self-consistency

In self-consistency, we generate multiple answers to the same question and pick the answer that is repeated the most across these occurrences. This is particularly valuable for factual questions.

In [ ]:
answers_template = """
Context:
{context}
===
Using the context above generate {num_answers} distinct answers to the following question:
Question:
{question}.

Arrange your answers in numbered bullet points.
Present only the answers in bullet points.
"""

Here is an extract from the [Tesla 2022 10-K](https://www.sec.gov/Archives/edgar/data/1318605/000095017023001409/tsla-20221231.htm) statement that will be used as context for this demonstration.

In [ ]:
tesla_annual_report_context ="""
In 2022, we recognized total revenues of $81.46 billion, respectively, representing an increase of $27.64 billion, compared to the prior year.
We continue to ramp production, build new manufacturing capacity and expand our operations to enable increased deliveries and deployments of our products and further revenue growth.
"""

In [ ]:
factual_question = "What was the increase in annual revenue in 2022 compared to 2021?"

In [ ]:
answers_prompt = mistral_prompt_template.format(
    prompt=answers_template.format(
        context=tesla_annual_report_context,
        question=factual_question,
        num_answers=3
    )
)

In [ ]:
response = lcpp_llm(
    prompt=answers_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False # do not return the prompt
)

In [ ]:
factual_answers = response["choices"][0]["text"].strip()

In [ ]:
print(factual_answers)

* The increase in annual revenue from 2021 to 2022 was $27.64 billion.
* In 2022, total revenues were $81.46 billion, which is an increase of $27.64 billion compared to the previous year.
* The company experienced a revenue growth of approximately $27.64 billion between 2021 and 2022.


In [ ]:
consistency_template = """
Here are {num_answers} answers to the question mentioned below:
Question:
{question}
Answers:
{answers}

Observe the answers mentioned above and choose the answer that occurs most.
Present only the most frequent solution in the following format.
Final Answer:
"""

In [ ]:
consistency_prompt = mistral_prompt_template.format(
    prompt=consistency_template.format(
        num_answers=3,
        question=factual_question,
        answers=factual_answers
    )
)

In [ ]:
response = lcpp_llm(
    prompt=consistency_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False # do not return the prompt
)

Llama.generate: prefix-match hit


In [ ]:
print(response["choices"][0]["text"].strip())

Final Answer: The increase in annual revenue from 2021 to 2022 was $27.64 billion.


## Tree-of-Thought

Tree-of-thought prompting is a generalization of chain-of-thought prompting where the model is prompted to take multiple reasoning paths. This forces the LLM into a deliberate reasoning mode.

In [ ]:
solutions_template = """
Generate {num_solutions} distinct solutions for the following problem:
Problem:
{problem}.
--

Consider the following factors in coming up with your solutions.
Factors:
{factors}

Present the {num_solutions} solutions in numbered bullet points. Present only the solutions.
"""

In [ ]:
climate_problem = "Reduce the impact of climate change on the occurrence of extreme events in the Earth's atmosphere."

climate_factors = """
1. Renewable Energy Transition
2. Reforestation
3. Sustainable Agricultural Practises
4. Carbon capture and storage
5. Climate-resilient infrastructure
6. Circular economy practises
"""

In [ ]:
solutions_prompt = mistral_prompt_template.format(
    prompt=solutions_template.format(
        num_solutions=3,
        problem=climate_problem,
        factors=climate_factors
    )
)

In [ ]:
response = lcpp_llm(
    prompt=solutions_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False # do not return the prompt
)

Llama.generate: prefix-match hit


In [ ]:
climate_solutions = response["choices"][0]["text"].strip()

In [ ]:
print(climate_solutions)

Solution 1:
- Accelerate the transition to renewable energy sources by investing in solar, wind, hydroelectric, and other clean energy technologies.
- Encourage governments and businesses to set ambitious targets for reducing greenhouse gas emissions.
- Implement policies that incentivize individuals and organizations to adopt renewable energy solutions.

Solution 2:
- Increase global reforestation efforts by planting new trees, protecting existing forests, and promoting sustainable forest management practices.
- Encourage the use of agroforestry systems, which combine agriculture with tree cultivation, to help sequester carbon and reduce deforestation.
- Support research into innovative approaches for afforestation (planting new forests in previously treeless areas) and reforestation (reestablishing forests on degraded or deforested land).

Solution 3:
- Implement climate-resilient infrastructure projects, such as green roofs, rainwater harvesting systems, and permeable pavements.
- E

In [ ]:
evaluation_template = """
For the following problem: {problem}, evaluate each solution in the following proposed solutions: \n{solutions}\n.
Analyze pros, cons, feasibility, and probability of success for each solution.
Present your evaluations of each solutions.
"""

In [ ]:
evaluations_prompt = mistral_prompt_template.format(
    prompt=evaluation_template.format(
        problem=climate_problem,
        solutions=climate_solutions
    )
)

In [ ]:
response = lcpp_llm(
    prompt=evaluations_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False # do not return the prompt
)

Llama.generate: prefix-match hit


In [ ]:
climate_proposal_evaluations = response["choices"][0]["text"].strip()

In [ ]:
print(climate_proposal_evaluations)

Solution 1: Transitioning to Renewable Energy Sources
Pros:
- Reduces greenhouse gas emissions from the power sector, which is a major contributor to climate change.
- Creates jobs and stimulates economic growth in the renewable energy industry.
- Decreases dependence on fossil fuels, reducing vulnerability to price volatility and supply disruptions.
Cons:
- High upfront costs for infrastructure development and implementation of renewable energy technologies.
- Intermittency issues with some renewable sources (e.g., solar and wind) may require backup power or energy storage solutions.
- Political challenges in implementing policies that incentivize the transition to renewable energy, particularly in countries heavily reliant on fossil fuel industries.
Feasibility: Renewable energy technologies have become increasingly cost-competitive with traditional fossil fuels and are continuing to improve in efficiency. Many countries have already set ambitious targets for reducing greenhouse gas 

In [ ]:
ranking_template = """
For the following problem: {problem}, rank the solutions presented in the following evaluations: \n{evaluations}\n.
Pick most promising solution and present implementation strategies and methods to handle potential obstacles for this solution.
"""

In [ ]:
ranking_prompt = mistral_prompt_template.format(
    prompt=ranking_template.format(
        problem=climate_problem,
        evaluations=climate_proposal_evaluations
    )
)

In [ ]:
response = lcpp_llm(
    prompt=ranking_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False # do not return the prompt
)

Llama.generate: prefix-match hit


In [ ]:
climate_proposal_rankings = response["choices"][0]["text"].strip()

In [ ]:
print(climate_proposal_rankings)

 Based on the evaluations provided, all three solutions - transitioning to renewable energy sources, reforestation and afforestation efforts, and climate-resilient infrastructure and circular economy practices - have their merits in reducing the impact of climate change on extreme events in the Earth's atmosphere. However, considering the current global context and the feasibility and probability of success for each solution, transitioning to renewable energy sources appears to be the most promising option.

Implementation Strategies:
1. Government Policies and Incentives: Governments can play a crucial role in implementing policies that incentivize the transition to renewable energy. This could include setting targets for reducing greenhouse gas emissions, providing subsidies or tax credits for renewable energy projects, and investing in research and development of new technologies.
2. Public-Private Partnerships: Collaboration between governments, private companies, and international